In [35]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

**Code Explanation:**




1.   ***Data Preparation:***

*   The dataset is prepared by selecting specific predictor variables: pl_orbsmax (semi-major axis), st_mass (stellar mass), and pl_orbeccen (eccentricity).
*   ***Kepler's Law Transformation:*** Since Kepler's Third Law relates orbital period to the semi-major axis raised to the power of 3/2, the code applies this transformation to pl_orbsmax (X['pl_orbsmax'] = X['pl_orbsmax'] ** (3 / 2)) to better align with theoretical expectations.
*   The data is then split into training and test sets to allow for independent evaluation on unseen data.



2.   ***Model Initialization:*** The code initializes several models with different approaches:

*   ***Linear Regression:*** Finds a linear relationship between predictors and the target variable (pl_orbper). It serves as a baseline, providing insight into linear trends in the data.

*   ***Polynomial Regression:*** Extends Linear Regression by adding polynomial terms (degree 2), allowing it to capture more complex, non-linear relationships.

*   ***Decision Tree Regressor:*** A non-linear model that splits data into branches based on conditions in the predictor variables, capturing intricate patterns but potentially overfitting without pruning.

*   ***Random Forest Regressor:*** An ensemble of decision trees that averages predictions from multiple trees trained on random subsets of data, improving accuracy and reducing overfitting.

*   ***Gradient Boosting Regressor:*** An ensemble method that builds trees sequentially, each focusing on correcting errors from previous trees. It’s powerful for capturing complex patterns and is effective when properly tuned.






3.   ***Cross-Validation (Folding):***

* The code uses 5-fold cross-validation to train and evaluate each model across multiple data splits.

* In 5-fold cross-validation, the data is divided into 5 subsets (folds). Each model is trained on 4 folds and tested on the remaining fold. This process repeats 5 times, with each fold serving as the test set once.

* Advantages: Cross-validation provides a more reliable assessment of model performance by reducing bias and ensuring that the model is tested on all data points. It also helps in identifying any overfitting, as each model must generalize across different parts of the dataset.

* The results (Mean Squared Error and R²) for each fold are averaged, providing a comprehensive measure of model accuracy and robustness.

4.   ***Model Training and Evaluation:*** Each model is trained and evaluated on the training and test sets:

*    ***Mean Squared Error (MSE)*** indicates the average squared difference between predicted and actual values, where lower values represent higher accuracy.

*    ***R² Score*** measures the proportion of variance in pl_orbper explained by the model, with higher values indicating better explanatory power.

* Each model's results on different folds (from cross-validation) are examined, and metrics like MSE and R² are printed for each fold to assess performance consistency.


5. ***Stacking Ensemble Model:***

*   ***Stacking*** is an ensemble technique that combines predictions from multiple models to improve accuracy. Here, each base model (Linear Regression, Polynomial Regression, Decision Tree, Random Forest, Gradient Boosting) provides predictions, which are then used as inputs to a meta-model (Random Forest Regressor).

* The stacking ensemble benefits from the strengths of each model, creating a more robust prediction by compensating for the individual weaknesses of each base model.

* The final stacking model is trained using the cross-validated predictions from the base models, allowing it to leverage multiple perspectives (linear, non-linear, and complex patterns) in its final prediction.

6. ***Final Evaluation and Visualization:***

* After training, the stacking model is evaluated on the test set to determine its performance relative to the individual models.

* A plot of actual vs. predicted values is generated, showing how closely the model’s predictions align with actual values. Ideally, points would lie along the "Perfect Fit" line, with deviations indicating prediction errors.

In [38]:
planet_data = pd.read_csv("planetary_data.csv")
cleaned_data = planet_data.dropna()
print("First few rows of the cleaned data:")
print(cleaned_data.head(10))

First few rows of the cleaned data:
         pl_name        hostname     pl_orbper  pl_orbsmax  pl_rade  \
30      51 Eri b          51 Eri  11688.000000    13.20000   13.400   
32      55 Cnc b          55 Cnc     14.651600     0.11340   13.900   
35      55 Cnc e          55 Cnc      0.736547     0.01544    1.875   
51      AF Lep b          AF Lep   8030.000000     8.40000   13.100   
52      AU Mic b          AU Mic      8.463080     0.06490    3.957   
53      AU Mic c          AU Mic     18.859690     0.11080    2.522   
62   BD+20 594 b       BD+20 594     41.685500     0.24100    2.578   
78  BD-14 3065 b    BD-14 3065 A      4.288973     0.06560   21.590   
82     Barnard b  Barnard's star      3.153300     0.02294    0.764   
90     CoRoT-1 b         CoRoT-1      1.508956     0.02752   16.700   

    pl_bmasse  pl_orbeccen  st_rad  st_mass  pl_eqt  st_teff  st_met  st_logg  
30   635.6600      0.45000    1.49     1.75   700.0   7295.0  -0.027     4.31  
32   263.9785      0.0

In [40]:
numDF = pd.DataFrame(cleaned_data[["pl_orbper","pl_orbsmax","pl_rade","pl_bmasse","pl_orbeccen","st_rad","st_mass","pl_eqt","st_teff","st_met","st_logg"]])
numDF.head(5)

,pl_orbper,pl_orbsmax,pl_rade,pl_bmasse,pl_orbeccen,st_rad,st_mass,pl_eqt,st_teff,st_met,st_logg
30,11688.000000,13.20000,13.400,635.6600,0.45000,1.49,1.75,700.0,7295.0,-0.027,4.31
32,14.651600,0.11340,13.900,263.9785,0.00000,0.94,0.91,700.0,5172.0,0.350,4.43
35,0.736547,0.01544,1.875,7.9900,0.05000,0.94,0.91,1958.0,5172.0,0.350,4.43
51,8030.000000,8.40000,13.100,1017.0509,0.24000,1.25,1.20,1400.0,6130.0,0.190,4.40
52,8.463080,0.06490,3.957,20.1200,0.00577,0.74,0.51,600.0,3678.0,0.230,4.40


In [42]:
numDF.describe()


,pl_orbper,pl_orbsmax,pl_rade,pl_bmasse,pl_orbeccen,st_rad,st_mass,pl_eqt,st_teff,st_met,st_logg
count,3842.000000,3842.000000,3842.000000,3842.000000,3842.000000,3842.000000,3842.000000,3842.000000,3842.00000,3842.000000,3842.000000
mean,101.627980,0.192924,4.405607,210.490577,0.037527,1.028563,0.954323,908.609058,5452.95139,0.019792,4.440060
std,1635.739138,1.098874,4.655811,2336.573452,0.104599,0.424115,0.256734,451.081034,763.16047,0.166169,0.222449
min,0.179719,0.005800,0.310000,0.037400,0.000000,0.120000,0.090000,96.000000,2566.00000,-0.920000,2.950000
25%,4.038349,0.048600,1.620000,3.392500,0.000000,0.790000,0.820000,570.000000,5107.25000,-0.060000,4.312500
50%,9.008195,0.080800,2.447000,7.050000,0.000000,0.950000,0.950000,818.500000,5623.50000,0.020000,4.470000
75%,21.796833,0.150200,3.990000,20.377500,0.000000,1.200000,1.080000,1156.000000,5946.00000,0.120000,4.570000
max,69000.000000,38.000000,32.600000,89700.000000,0.930000,6.300000,2.780000,4050.000000,10170.00000,0.560000,5.400000


In [ ]:

# Set the plot style
sb.set(style="whitegrid")

# Adjust the figure size for better visibility
fig, axes = plt.subplots(len(numDF.columns), 3, figsize=(15, 5 * len(numDF.columns)))

# Loop through each column in the DataFrame
for i, column in enumerate(numDF.columns):
    # Box plot with custom color
    sb.boxplot(data=numDF, x=column, ax=axes[i, 0], color="skyblue")
    axes[i, 0].set_title(f"Box Plot of {column}")

    # Histogram with KDE and custom color
    sb.histplot(data=numDF, x=column, kde=True, ax=axes[i, 1], color="lightgreen")
    axes[i, 1].set_title(f"Histogram of {column}")

    # Violin plot with custom color
    sb.violinplot(data=numDF, x=column, ax=axes[i, 2], color="salmon")
    axes[i, 2].set_title(f"Violin Plot of {column}")

# Adjust layout to prevent overlap and improve spacing
plt.tight_layout()
plt.show()


*    ***pl_orbper (Orbital Period):***
The distribution is heavily skewed to the left with many outliers at higher values. Most data points have low orbital periods, suggesting that long-period planets are rare in this dataset.

*   ***pl_orbsmax (Orbit Semi-Major Axis):***
Similar to pl_orbper, this variable is left-skewed with a high concentration of values at lower ranges and several outliers. This indicates that most planets have relatively close orbits to their stars.

*   ***pl_rade (Planet Radius):***
Shows a positively skewed distribution with a few larger outliers. The majority of planets have a radius within a certain small range, with a long tail of larger planets.

*   ***pl_bmasse (Planet Mass):***
Extremely right-skewed with significant outliers. Most planets have a low mass, but there are a few high-mass planets that deviate from the general distribution.

*   ***pl_orbeccen (Eccentricity):***
Eccentricity values are clustered around zero, indicating that most orbits are nearly circular, with a small number of planets having more eccentric orbits.

*   ***st_rad (Stellar Radius):***
Shows a slightly right-skewed distribution. The majority of stars have radii close to the solar radius, with a few larger stars as outliers.

*   ***st_mass (Stellar Mass):***
Concentrated around 1 solar mass with some outliers. This suggests that most stars are similar to the Sun in mass, with a few exceptions.

*   ***pl_eqt (Equilibrium Temperature):***
Distribution is left-skewed, with most planets having moderate temperatures and a few outliers at higher temperatures. This likely reflects the range of distances from their stars.

*   ***st_teff (Stellar Effective Temperature):***
This variable is also right-skewed, with a peak around average stellar temperatures and a few high-temperature outliers, indicating a variety in stellar types.

*   ***st_met (Stellar Metallicity):***
Fairly symmetric distribution centered around zero, with minimal skewness. This suggests a balanced range of stars with metal-poor and metal-rich compositions.

*   ***st_logg (Stellar Surface Gravity):***
Slightly skewed towards higher values, with a peak around typical values for surface gravity. Most stars cluster around a similar range, with few outliers.

In [ ]:
sb.pairplot(data = numDF)

***pl_orbsmax (Orbit Semi-Major Axis):***

*   ***Observation:*** There is a strong, non-linear positive correlation between pl_orbsmax and pl_orbper. As the semi-major axis increases, the orbital period also increases, following a curved pattern.
*   ***Theoretical Support:*** According to Kepler’s third law of planetary motion, the orbital period of a planet is proportional to the 3/2 power of the semi-major axis. This law explains why the semi-major axis is the most significant predictor of orbital period; larger orbits naturally result in longer periods due to the increased distance traveled.







***pl_bmasse (Planet Mass):***

*   ***Observation:*** There is no clear correlation between pl_bmasse and pl_orbper. Most values of pl_bmasse are clustered at low values of pl_orbper, with a few outliers showing very high planet masses without affecting the period.
*   ***Theoretical Support:*** In gravitational systems, the mass of the planet has minimal impact on the orbital period because the primary force acting on the planet is the star’s gravity. The planet’s mass only has a minor effect unless it is comparable to the star’s mass (e.g., in binary systems), which is not the case here.







***pl_orbeccen (Eccentricity):***

*   ***Observation:*** Eccentricity does not show a clear pattern with pl_orbper. The scatter is random, suggesting eccentricity does not significantly influence the period.
*   ***Theoretical Support:*** Eccentricity affects the shape of the orbit, but not the average orbital period in a straightforward way. While more eccentric orbits might change the planet's speed at different points in the orbit, the overall period remains largely determined by the semi-major axis, according to Kepler’s laws.







***st_rad (Stellar Radius):***

*   ***Observation:*** There is no clear trend between st_rad and pl_orbper. The values are mostly concentrated at low stellar radii.
*   ***Theoretical Support:*** Stellar radius does not directly influence the orbital period. The radius affects factors like stellar luminosity and potential habitable zones but does not change the gravitational force felt by the planet, which is more dependent on stellar mass and distance.







***st_mass (Stellar Mass):***

*   ***Observation:*** There is a slight, weak correlation between st_mass and pl_orbper, with higher stellar masses potentially resulting in shorter orbital periods.
*   ***Theoretical Support:*** Stellar mass has a theoretical influence on orbital period since a more massive star exerts a stronger gravitational pull, which could lead to faster orbits for nearby planets. However, in this dataset, the effect appears weak, possibly because the semi-major axis dominates the relationship with orbital period.







***pl_eqt (Equilibrium Temperature):***

*   ***Observation:*** No clear correlation exists between pl_eqt and pl_orbper.
*   ***Theoretical Support:*** Equilibrium temperature is primarily influenced by the star’s temperature and distance from the planet. It indirectly depends on the semi-major axis but does not influence the orbital mechanics or period itself.







***st_teff (Stellar Effective Temperature):***

*   ****Observation:*** No distinct pattern exists between st_teff and pl_orbper.
*   ***Theoretical Support:*** Stellar effective temperature affects the star’s luminosity and potential habitability zones but does not impact the gravitational pull that determines orbital period.

***st_met (Stellar Metallicity):***
*   ***Observation:*** There is no clear pattern or correlation between st_met and pl_orbper. Most points are clustered near low values of pl_orbper, and the values of st_met are spread out without any visible trend.
*   ***Theoretical Support:*** Stellar metallicity refers to the abundance of elements heavier than hydrogen and helium in a star. While metallicity can affect the formation and composition of planets, it does not impact the gravitational forces or mechanics that determine the orbital period. Thus, it’s expected to have minimal influence on pl_orbper.


***st_logg (Stellar Surface Gravity):***
*   ***Observation:*** Similar to st_met, there is no clear relationship between st_logg and pl_orbper. Points are dispersed without any noticeable trend or pattern, indicating that variations in stellar surface gravity do not correlate with orbital period.
*   ***Theoretical Support:*** Stellar surface gravity reflects the gravitational pull at the star’s surface. While it indirectly relates to stellar mass and radius, it doesn’t directly influence the orbital period of planets. Orbital period is primarily a function of the semi-major axis and stellar mass, not surface gravity.



***Conclusion***

In summary, the variables most relevant for predicting pl_orbper are:

*   ***Primary Predictor:***  pl_orbsmax (Orbit Semi-Major Axis)
*   ***Secondary Predictor:***  st_mass (Stellar Mass)
*   ***Minor Influence:***  pl_orbeccen (Orbital Eccentricity)

These three variables, particularly pl_orbsmax, align well with theoretical principles and also have been visualised here. All other variables have negligible impact and can likely be excluded from the predictive model without affecting accuracy.



In [ ]:
numDF.corr()

According to the correlation matrix, the best predictors for pl_orbper are:

*   ***Primary Predictor:***  pl_orbsmax (Orbit Semi-Major Axis) – High correlation (0.93) and aligns with theoretical principles.

*   ***Secondary Predictors:*** pl_orbeccen (Orbital Eccentricity) and st_mass (Stellar Mass) – Moderate (0.17) and weak (0.06) correlations, respectively, with slight influence on orbital period.

All other variables have negligible correlations and lack theoretical justification for affecting pl_orbper. Therefore, they can likely be excluded from a predictive model without impacting its accuracy.

In [ ]:
# Heatmap of the Correlation Matrix
f = plt.figure(figsize=(12, 12))
sb.heatmap(numDF.corr(), vmin = -1, vmax = 1, annot = True, fmt = ".2f")

This heatmap is just the visual representation of what we just saw in the correaltion matrix and hence it justifies the use of pl_orbsmax, pl_orbeccen, and st_mass as our predictors

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Assuming numDF is available and contains 'pl_orbsmax', 'st_mass', 'pl_orbeccen', and 'pl_orbper'
X = numDF[['pl_orbsmax', 'st_mass', 'pl_orbeccen']].copy()
y = numDF['pl_orbper']

# Apply transformation to pl_orbsmax for Kepler's law
X['pl_orbsmax'] = X['pl_orbsmax'] ** (3 / 2)

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

# Store predictions for stacking
stack_train_predictions = np.zeros((len(X), len(models)))  # To store training predictions for each fold and model
stack_test_predictions = np.zeros((len(X_test), len(models)))  # Store test predictions for final ensemble

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/{kf.n_splits}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Keep track of predictions from each model for each fold
    for model_index, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)

        # Predict on validation set and store in stacking matrix
        val_pred = model.predict(X_val)
        stack_train_predictions[val_idx, model_index] = val_pred

        # Aggregate test predictions across all folds for averaging
        test_pred = model.predict(X_test)
        stack_test_predictions[:, model_index] += test_pred / kf.n_splits  # Average over folds

        # Calculate and print metrics for this fold and model
        fold_mse = mean_squared_error(y_val, val_pred)
        fold_r2 = r2_score(y_val, val_pred)
        print(f"{name} - Fold {fold + 1} MSE: {fold_mse}, R²: {fold_r2}")

# Convert stack_train_predictions and stack_test_predictions into DataFrames for readability
stack_train_df = pd.DataFrame(stack_train_predictions, columns=models.keys())
stack_test_df = pd.DataFrame(stack_test_predictions, columns=models.keys())

# Final Stacking Ensemble Model using the predictions from each base model
stacking_ensemble = StackingRegressor(
    estimators=[(name, model) for name, model in models.items()],
    final_estimator=RandomForestRegressor(n_estimators=50, random_state=42),
    passthrough=True
)

# Fit the stacking ensemble on the entire stack_train_predictions
stacking_ensemble.fit(stack_train_df, y)

# Make predictions with the stacking ensemble
train_pred_stacking = stacking_ensemble.predict(stack_train_df)
test_pred_stacking = stacking_ensemble.predict(stack_test_df)

# Evaluate the stacking ensemble model
train_mse_stacking = mean_squared_error(y, train_pred_stacking)
train_r2_stacking = r2_score(y, train_pred_stacking)
print("\nFinal Stacking Ensemble Train MSE:", train_mse_stacking)
print("Final Stacking Ensemble Train R²:", train_r2_stacking)

# Assuming y_test is available for comparison with test predictions
test_mse_stacking = mean_squared_error(y_test, test_pred_stacking)
test_r2_stacking = r2_score(y_test, test_pred_stacking)
print("Final Stacking Ensemble Test MSE:", test_mse_stacking)
print("Final Stacking Ensemble Test R²:", test_r2_stacking)

# Plot Actual vs Predicted for Stacking Ensemble
plt.figure(figsize=(8, 6))
sb.scatterplot(x=y_test, y=test_pred_stacking, color='purple', alpha=0.6, s=50, label='Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Fit')
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Stacking Ensemble - Actual vs Predicted Values (Test Set)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


*   ***High Predictive Accuracy in Most Cases:***
The Stacking Ensemble achieves a high R² score of 0.93 on the training set and 0.83 on the test set, demonstrating its strong ability to capture relationships within the data.
This high accuracy makes it effective for generating reliable predictions within the typical range, which is valuable as a starting point for further refinement.

*   ***Robustness through Ensemble Learning:***
By combining multiple models (Linear Regression, Decision Tree, Random Forest, and Gradient Boosting), the Stacking Ensemble leverages the strengths of each model while reducing individual weaknesses.
This approach leads to a more generalized model that performs well across varied data subsets, making it more adaptable and robust than any single model alone.

*   ***Validated through Cross-Validation for Reliability:***
The use of K-Fold Cross-Validation ensures that the model’s performance is not biased by any single data split, making it more reliable and robust across varied data samples.
This cross-validation approach builds confidence that the model’s high accuracy is consistent and dependable.

*   ***